# 0.1.4: Build GBIF trait maps

The final step before training models using Earth observation (EO) data is to link the TRY trait data with the GBIF species observations and then grid them. In this way, we can have matching trait rasters to be paired with our EO data.

## Imports and config

In [1]:
from pathlib import Path

import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import dask_geopandas as dgpd
import geopandas as gpd
import numpy as np
import pandas as pd
from src.conf.conf import get_config
from src.conf.environment import log

%load_ext autoreload
%autoreload 2

# Display all columns when printing a pandas DataFrame
pd.set_option("display.max_columns", None)

cfg = get_config()

cluster = LocalCluster(dashboard_address=":39143")
client = Client(cluster)

## Load GBIF and filter by PFT

Let's load the GBIF data and select only the "Tree" PFT.

In [2]:
def filter_pft(df: pd.DataFrame, pft_set: str, pft_col: str = "pft") -> pd.DataFrame:
    pfts = pft_set.split("_")
    if not any(pft in ["Shrub", "Tree", "Grass"] for pft in pfts):
        raise ValueError(f"Invalid PFT designation: {pft_set}")

    return df[df[pft_col].isin(pfts)]


gbif = (
    dd.read_parquet(Path(cfg.gbif.interim.dir, cfg.gbif.interim.subsampled))
    .pipe(filter_pft, "Tree")
    .repartition(npartitions=60)
    .sample(frac=0.01)
    .set_index("speciesname")
)

## Load TRY filtered mean trait data

In [3]:
mn_traits = (
    dd.read_parquet(Path(cfg.trydb.interim.dir, cfg.trydb.interim.filtered))
    .repartition(npartitions=60)
    .set_index("speciesname")
)

## Link mean trait values with GBIF data

In [4]:
merged = gbif.join(mn_traits, how="inner").reset_index()

In [14]:
# compute the number of unique indices
print(
    f"Pct matched species: {merged.index.nunique().compute() / gbif.index.nunique():.2%}"
)

Pct matched species: 61.89%


## Rasterize merged trait values

In [5]:
def global_grid_data(
    df: pd.DataFrame,
    traits: list[str],
    lon: str = "decimallongitude",
    lat: str = "decimallatitude",
    res: int | float = 0.5,
):
    """
    Modified from:
    https://sojwolf.github.io/iNaturalist_traits/Chapter_6_Compare_trait_maps_sPlot_iNat.html#grid-mean-trait-values-at-different-resolutions
    """
    # convert resolution in degrees into step size
    step = int((360 / res) + 1)

    bins_x = np.linspace(-180, 180, step)
    bins_y = np.linspace(-90, 90, int(((step - 1) / 2) + 1))

    # group latitude and longitude coordinates into bins
    df["x_bin"] = df.map_partitions(
        pd.cut, lon, bins=bins_x, meta=pd.Series(dtype="category", name="x_bin")
    )
    df["y_bin"] = df.map_partitions(
        pd.cut, lat, bins=bins_y, meta=pd.Series(dtype="category", name="y_bin")
    )

    df = df.dropna(subset=["x_bin", "y_bin"])
    
    # raster coordinates are in center of raster cell
    df["x_bin"] = df["x_bin"].map(lambda x: ((x.left + x.right) / 2))
    df["y_bin"] = df["y_bin"].map(lambda x: ((x.left + x.right) / 2))

    # group by 'x_bin' and 'y_bin' and calculate the mean of all variables
    grouped_df = df.groupby(["x_bin", "y_bin"])[traits].mean().reset_index()

    return grouped_df.compute()

In [7]:
res = 0.5
lon = "decimallongitude"
lat = "decimallatitude"
traits = [col for col in merged.columns if col.startswith("X")]

# convert resolution in degrees into step size
step = int((360 / res) + 1)

bins_x = np.linspace(-180, 180, step)
bins_y = np.linspace(-90, 90, int(((step - 1) / 2) + 1))


In [28]:
df = merged.copy()
# group latitude and longitude coordinates into bins
# df["x_bin"] = df.map_partitions(
#     pd.cut, df[lon], bins=bins_x, meta=pd.Series(dtype="category", name="x_bin")
# )
# df["y_bin"] = df.map_partitions(
#     pd.cut, df[lat], bins=bins_y, meta=pd.Series(dtype="category", name="y_bin")
# )

df["x_bin"] = df[lon].map_partitions(pd.cut, bins=bins_x, meta=pd.Series(dtype="category", name="x_bin"))
df["y_bin"] = df[lat].map_partitions(pd.cut, bins=bins_y, meta=pd.Series(dtype="category", name="y_bin"))

In [19]:
binned = df.compute()

In [29]:
# df = df.dropna(subset=["x_bin", "y_bin"])

# raster coordinates are in center of raster cell
df["x_bin_c"] = df["x_bin"].map(
    lambda x: ((x.left + x.right) / 2), meta=pd.Series(dtype="category", name="x_bin_c")
)
df["y_bin_c"] = df["y_bin"].map(
    lambda x: ((x.left + x.right) / 2), meta=pd.Series(dtype="category", name="y_bin_c")
)

df = df.drop(columns=["x_bin", "y_bin"])
# group by 'x_bin' and 'y_bin' and calculate the mean of all variables
grouped_df = df.groupby(["x_bin_c", "y_bin_c"])[traits].mean().reset_index()

grouped_df = grouped_df.compute()

/home/dl1070/micromamba/envs/traits/lib/python3.12/site-packages/dask_expr/_groupby.py:1542: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  self._meta = self.obj._meta.groupby(


In [33]:
grouped_df.dropna()

,x_bin_c,y_bin_c,X4,X6,X11,X13,X14,X15,X18,X21,X26,X27,X46,X47,X50,X55,X78,X95,X138,X144,X145,X146,X163,X169,X223,X224,X237,X281,X282,X289,X1080,X3112,X3113,X3114,X3120
3269,-175.25,-21.25,0.555270,3.143115,12.305853,457.361592,20.393410,1.216815,15.116824,0.155300,153.077884,5.394619,0.301134,0.368206,1.946444,720.607358,2.679780,93.401972,136.575768,129.193032,5.505791,26.121147,2.236319,16.979213,25.289191,2.620529,6.237009,39.290614,365.972825,831.696003,1916.782144,8542.602348,8517.929687,16349.308806,2.871731
13359,-161.25,68.75,0.306433,0.146528,13.688875,520.261222,19.833073,1.027163,0.300316,0.193642,0.269588,1.343754,0.181581,0.406500,1.504970,6.518777,-3.970987,59.591231,13.736337,13.408349,0.674482,31.833767,0.016404,477.430135,41.653959,1.355625,4.493998,19.913139,224.364773,408.701330,3231.883424,109.081570,116.189704,143.409645,2.703318
15979,-157.75,21.75,0.620640,1.410433,9.740822,463.359594,17.005442,1.211155,10.825915,0.115526,13.781408,4.061588,0.364955,0.391651,1.755757,189.025089,0.790468,90.948280,438.757288,100.325707,2.834415,39.248728,0.581193,27.400398,29.834242,2.620119,5.074030,35.216828,451.281260,986.332409,2500.627206,2344.552193,1167.729036,2290.745837,2.424438
16866,-156.75,21.25,0.648778,2.319221,8.566886,442.928369,16.367356,1.406025,11.249575,0.122176,46.551881,5.105120,0.386553,0.410172,1.990783,287.241672,1.533728,92.961718,615.724812,194.998554,5.049453,31.696263,0.737676,19.104674,22.017072,2.493095,5.564563,33.809561,438.014939,937.166588,1751.085313,3447.847238,1825.581697,9025.432145,2.233543
17504,-155.75,18.75,0.478463,0.794792,31.014784,479.561032,38.419323,1.888614,7.250706,0.651814,33.111878,3.588892,0.141178,0.296574,1.711009,215.208053,-0.457784,76.152081,43.743874,54.527767,9.244079,13.069040,0.980812,19.359709,56.066937,1.391154,6.296776,46.895420,377.524137,740.466394,1184.294616,9693.542014,6115.364532,6294.918989,3.852160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515156,175.25,-39.25,0.521271,1.610957,10.325707,477.064797,14.895688,1.082890,7.095267,0.330091,102.233242,4.443446,0.365771,0.337638,1.805076,227.422011,-0.811196,83.581282,3504.744562,78.880713,2.228554,39.401989,0.663778,128.487248,31.656015,7.476535,7.875111,27.725480,474.889044,904.059487,2830.791562,1296.636494,1456.989556,1945.155721,3.315672
515187,175.75,-40.25,0.521067,1.122285,14.406432,463.754455,20.479786,1.427804,8.666326,0.240029,45.697759,4.219196,0.261010,0.309489,1.611862,225.435098,0.016807,83.992623,4200.742679,92.472565,3.282736,27.718582,0.730878,86.641348,36.450174,3.816865,6.943386,32.223362,427.943850,832.514254,3322.949250,2303.415868,1884.548677,2726.333759,3.873424
515374,175.75,66.75,0.621460,1.189967,13.165759,470.669985,18.995897,1.746837,10.140136,0.178917,5.077258,3.804276,0.206947,0.366868,1.432214,518.564147,-0.966383,75.604697,1584.241150,46.666829,4.728433,29.285670,1.525033,147.766832,34.657260,1.858460,7.180175,29.992033,509.240514,843.172907,4949.432389,6973.681741,761.561477,3313.887073,2.983286
516069,176.75,-37.75,0.583219,1.158736,15.927349,466.814262,18.508769,1.313584,9.734221,0.079745,220.197330,8.397437,0.211833,0.346254,1.188524,353.470355,-0.110024,90.254098,70.193354,102.772185,4.070033,25.644594,1.141719,24.278078,26.474992,4.252726,9.561295,24.764884,430.590397,857.790715,1610.844254,5038.089406,3107.725877,4244.667050,3.353167


In [6]:
cols = [col for col in merged.columns if col.startswith("X")]
grid_data = global_grid_data(merged.copy(), cols)

ValueError: Metadata inference failed in `map`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
AttributeError("'float' object has no attribute 'left'")

Traceback:
---------
  File "/home/dl1070/micromamba/envs/traits/lib/python3.12/site-packages/dask/dataframe/utils.py", line 195, in raise_on_meta_error
    yield
  File "/home/dl1070/micromamba/envs/traits/lib/python3.12/site-packages/dask_expr/_expr.py", line 3983, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dl1070/micromamba/envs/traits/lib/python3.12/site-packages/dask/utils.py", line 1241, in __call__
    return getattr(__obj, self.method)(*args, **kwargs)
     

In [12]:
client.close()
cluster.close()